<a href="https://colab.research.google.com/github/Sakinat-Folorunso/OOU_CSC309_Artificial_Intelligence/blob/main/notebooks/CSC309_Week11_MultiAgent_CA3_Student_Centred.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CSC309 – Artificial Intelligence  
**Week 11 Lab:** Multi‑Agent Systems — Boids, Ant Foraging, or Message Passing (CA3)

**Instructor:** Dr Sakinat Folorunso  
**Mode:** Student‑centred, hands‑on in Google Colab

> Every code cell is commented line‑by‑line so you can follow the logic precisely.

## How to use this notebook
1. Start with the **Group Log** and **Do Now**.  
2. Run the **Setup** cell once.  
3. Work through **Tasks**. Edit only cells marked **`# TODO(Student)`**.  
4. Use **Quick Checks** to test your understanding.  
5. Finish with the **Reflection**. If you finish early, try the **Extensions**.

In [ ]:
#@title 🧑🏽‍🤝‍🧑🏾 Group Log (fill before you start)
# The '#@param' annotations create form fields in Colab for easy input.

group_members = "Type names here"  #@param {type:"string"}  # Names of teammates
roles_notes = "Driver/Navigator, decisions, questions"  #@param {type:"string"}  # Short working notes

print("👥 Group:", group_members)        # Echo the group list for confirmation
print("📝 Notes:", roles_notes)          # Echo the notes so they're preserved in output

### Learning Objectives
- Observe **emergent behaviour** from simple local rules.  
- Coordinate agents using **message passing**.

In [ ]:
#@title 🔧 Setup
# We use NumPy for math, Matplotlib for plots, and NetworkX for the message‑passing graph.

import sys, subprocess                                           # For potential installs
def pip_install(pkgs):
    for p in pkgs:
        try: __import__(p.split("==")[0])                        # Try import
        except Exception:
            subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", p])
pip_install(["numpy", "matplotlib", "networkx"])                 # Ensure these libs are present

import numpy as np                                               # Vectors and arrays
import matplotlib.pyplot as plt                                  # Visualisation
import networkx as nx                                            # Graphs for message passing
from collections import defaultdict, deque                       # Mailbox and frontier structures

print("✅ Setup complete for Week 11.")

In [ ]:
#@title 🐦 Boids (flocking) — fully commented

np.random.seed(0)                                                # Fix seed for reproducible behaviour
N = 40                                                           # Number of boids
X = np.random.rand(N, 2) * 10                                    # Positions (uniform in a 10x10 square)
V = np.random.randn(N, 2) * 0.1                                  # Initial velocities (small random)

def step_boids(X, V, dt=0.1):
    """Advance the boids simulation by one step with simple rules."""
    neighbor_r = 1.5                                             # Neighborhood radius
    sep_r = 0.5                                                  # Separation radius
    cohesion = 0.01                                              # Strength toward center of mass
    align = 0.05                                                 # Strength toward average velocity
    separate = 0.05                                              # Strength pushing away from close neighbors
    limit = 0.3                                                  # Max speed

    X2, V2 = X.copy(), V.copy()                                  # Work on copies to avoid in‑place surprises
    for i in range(len(X)):                                      # Update each boid 'i'
        diffs = X - X[i]                                         # Vector differences to all others
        dists = np.linalg.norm(diffs, axis=1) + 1e-6             # Euclidean distances (avoid zero)
        neigh = (dists < neighbor_r) & (dists > 0)               # Boolean mask for neighbors

        if np.any(neigh):                                        # If boid has neighbors
            center = X[neigh].mean(axis=0)                       # Cohesion: average neighbor position
            V2[i] += cohesion * (center - X[i])                  # Nudge velocity toward center of mass

            V2[i] += align * (V[neigh].mean(axis=0) - V[i])     # Alignment: match average neighbor velocity

            close = diffs[(dists < sep_r) & (dists > 0)]         # Boids too close
            if len(close):
                V2[i] -= separate * close.sum(axis=0)            # Separation: push away from close neighbors

        speed = np.linalg.norm(V2[i])                            # Compute speed
        if speed > limit:                                        # Enforce a speed limit
            V2[i] = V2[i] / speed * limit                        # Scale velocity to the limit

    X2 += V2 * dt                                                # Integrate positions X <- X + V*dt
    X2 = np.mod(X2, 10)                                          # Toroidal (wrap‑around) boundary
    return X2, V2                                                # Return updated positions and velocities

for _ in range(200):                                             # Run 200 steps
    X, V = step_boids(X, V)                                      # Advance the simulation

plt.scatter(X[:,0], X[:,1])                                      # Plot final positions
plt.title("Boids positions after 200 steps")                     # Figure title
plt.show()                                                       # Display the plot

In [ ]:
#@title 🐜 Ant Foraging (skeleton with comments)

np.random.seed(1)                                                # Fix seed
N = 40                                                           # Grid size N x N
pheromone = np.zeros((N, N), dtype=float)                        # Pheromone intensity grid
nest = (N//2, N//2)                                              # Nest in the center
food = [(5, 30), (32, 7)]                                        # Two food locations
world = np.zeros((N, N), dtype=int)                              # 0=empty; 2=nest; 3=food
world[nest] = 2                                                  # Mark nest cell
for f in food: world[f] = 3                                      # Mark food cells

class Ant:
    def __init__(self, pos):
        self.pos = pos                                           # Current position
        self.has_food = False                                    # Whether carrying food

    def step(self):
        # TODO(Student): Implement movement biased by pheromone and simple pickup/drop rules.
        # Hints:
        # - When NOT carrying food, prefer moving toward higher pheromone and random exploration.
        # - When ON a food cell, set 'has_food=True' and start heading home.
        # - When carrying food, deposit pheromone and head toward the nest coordinate.
        # - When at the nest with food, drop it (has_food=False).
        pass

ants = [Ant(nest) for _ in range(50)]                             # Create a swarm of ants

def evaporate(rate=0.95):
    global pheromone
    pheromone *= rate                                             # Exponential decay of pheromone

for t in range(200):                                              # Run 200 steps
    for a in ants: a.step()                                       # Update each ant
    evaporate()                                                   # Evaporate pheromone

plt.imshow(pheromone, cmap="viridis")                             # Visualize pheromone field
plt.title("Pheromone after 200 steps (tune your rules!)")         # Title
plt.colorbar()                                                    # Color scale
plt.show()                                                        # Show plot

In [ ]:
#@title 📨 Message Passing — distributed search (skeleton with comments)

G = nx.grid_2d_graph(5, 5)                                       # 5x5 grid graph as a simple network
target = (4, 4)                                                  # Target node to find
mailbox = defaultdict(deque)                                     # Per‑agent mailboxes

class Agent:
    def __init__(self, name, start):
        self.name = name                                         # Agent identifier
        self.frontier = deque([start])                            # Nodes to explore next
        self.visited = set([start])                               # Nodes already visited
        self.messages_sent = 0                                    # Count of messages sent

    def step(self, peers):
        while mailbox[self.name]:                                 # Process all incoming messages
            msg = mailbox[self.name].popleft()                    # Pop one message
            # TODO(Student): Merge frontier/visited info from peers (e.g., skip already seen nodes)

        if self.frontier:                                         # If there is something to explore
            s = self.frontier.popleft()                           # Take a node from the frontier
            if s == target:                                       # If we've found the target
                for p in peers:                                   # Inform all peers
                    mailbox[p].append(("found", s))               # Send 'found' message
                    self.messages_sent += 1                       # Count the message
                return True                                       # Report success
            for n in G.neighbors(s):                              # Otherwise, expand neighbors
                if n not in self.visited:                         # Only add unseen nodes
                    self.visited.add(n)                           # Mark as seen
                    self.frontier.append(n)                       # Add to frontier
                    for p in peers:                               # Tell peers you saw 'n'
                        mailbox[p].append(("seen", n))            # Broadcast 'seen' message
                        self.messages_sent += 1                   # Count message
        return False                                              # Not found yet

A = Agent("A", (0,0)); B = Agent("B", (0,1))                      # Two agents starting nearby
for t in range(50):                                               # Run up to 50 steps
    if A.step(["B"]) or B.step(["A"]):                            # Alternate steps; stop if found
        break
print("Messages sent: A =", A.messages_sent, "B =", B.messages_sent)  # Quick comms stat

### **CA3 Deliverables**
- Group demo (GIF/screencast acceptable) showing emergent behaviour or faster discovery via messages.  
- **Design log** with parameters, observations, and metrics.